In [ ]:
import sys
sys.path.append("../") 

In [22]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [ ]:
seed = 42
torch.manual_seed(seed) 
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 
np.random.seed(seed)  
random.seed(seed) 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 2

# 1253 target
weight = 50
limit = 0.3
sub = 0.022

lr = 4e-3
h_hyper_prmts["convlayers1"] = {"in_channels": 3824, "out_channels": 2048, "use_bn": False, "drop_rate": 0.9}
h_hyper_prmts["convlayers1"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0}
h_hyper_prmts["convlayers12"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0}
h_hyper_prmts["convlayers14"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0}
h_hyper_prmts["convlayers13"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0}
h_hyper_prmts["convlayers131"] = {"in_channels": 128, "out_channels": 64, "use_bn": False, "drop_rate": 0}


l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":64, "use_bn":False, "drop_rate":0}
l_hyper_prmts["linerlayer42"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0}
l_hyper_prmts["linerlayer421"] = {"in_channels":32, "out_channels":16, "use_bn":False, "drop_rate":0}
l_hyper_prmts["linerlayer422"] = {"in_channels":16, "out_channels":8, "use_bn":False, "drop_rate":0}
l_hyper_prmts["linerlayer423"] = {"in_channels":8, "out_channels":3, "use_bn":True, "drop_rate":0}



hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
def loss_bs_matrix(outs, hg, device, weight):

    H = hg.H.to_dense().to(device)
    outs = outs.to(device)

    X_ = outs.t().unsqueeze(-1)
    H_ = H.unsqueeze(0)
    xweight = H.sum(dim=0)
    mid = X_.mul(H_)
    sum = (mid * (1 / xweight)).sum()
    sub = (mid + (1 - H)).prod(dim=1).sum()
    loss_1 = sum - sub

    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2

    return loss, loss_1, loss_2

In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [27]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/pubmed")
edges, _ = G.e
G.num_e,G.num_v

(7523, 3824)

In [28]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0, inplace=False)
        (theta): Linear(in_features=2048, out_features=1024, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0, inplace=False)
        (theta): Linear(in_features=1024, out_features=512, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0, inplace=False)
        (theta): Linear(in_features=512, out_features=256, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0, inplace=False)
        (theta): Linear(in_features=256, out_features=128, bias=True)
      )
      (4): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0, inplace=False)
        (theta): Linear(in_features=128, out_features=64, bias=True)
      )
    )
  )
  (1): ReLU()
  (2): Linear(in_features=64, ou

In [29]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [ ]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=lr, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(40000):
    if hgnn_trainer.weight > limit:
        hgnn_trainer.weight = hgnn_trainer.weight - sub
    elif hgnn_trainer.weight < limit:
        hgnn_trainer.weight = limit
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    if loss_1 < 1150 and loss_2 < 0.01:
        break
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 35.04520263671875
                , loss1: 116.8171875
                , loss2: 4.5778354979120196e-05
                , weight: 0.3
in 10 epoch, average loss: 1014.62626953125
                , loss1: 1196.49365234375
                , loss2: 655.678076171875
                , weight: 0.3
in 20 epoch, average loss: 409.15205078125
                , loss1: 1198.2732421875
                , loss2: 49.670065307617186
                , weight: 0.3
in 30 epoch, average loss: 372.449755859375
                , loss1: 1199.83818359375
                , loss2: 12.498297119140625
                , weight: 0.3
in 40 epoch, average loss: 362.21572265625
                , loss1: 1195.4662109375
                , loss2: 3.5758567810058595
                , weight: 0.3
in 50 epoch, average loss: 358.88564453125
                , loss1: 1192.5267578125
                , loss2: 1.1275565147399902
                , weight: 0.3
in 60 epoch, average loss: 357.7085205078125
    

In [66]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

1157

In [67]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)

tensor([1274., 1275., 1275.], device='cuda:1', grad_fn=<SumBackward1>)
